💡 Cambia i settings della macchina in T4 per usare la GPU e ridurre i tempi di elaborazione

In [ ]:
from google.colab import drive
import os
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!git clone https://github.com/SesameAILabs/csm
# prompt: change directory on csm
%cd csm
!pip install -r requirements.txt

Cloning into 'csm'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 80 (delta 39), reused 7 (delta 7), pack-reused 28 (from 1)
Receiving objects: 100% (80/80), 36.07 KiB | 535.00 KiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/csm
  Cloning https://github.com/SesameAILabs/silentcipher (to revision master) to /tmp/pip-install-2a2vbv01/silentcipher_d5a8b9c574184c32a0d1bd3c51f1e4d5
  Running command git clone --filter=blob:none --quiet https://github.com/SesameAILabs/silentcipher /tmp/pip-install-2a2vbv01/silentcipher_d5a8b9c574184c32a0d1bd3c51f1e4d5
  Resolved https://github.com/SesameAILabs/silentcipher to commit d46d7d0893a583d8968ab3a6626e2289faec9152
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━

In [ ]:
from huggingface_hub import snapshot_download, login
from google.colab import userdata

# You need to add your Hugging Face token here to access gated models
# Get your token from https://huggingface.co/settings/tokens
HF_TOKEN = userdata.get('hf')  # Replace with your actual token

# Login to Hugging Face Hub
login(token=HF_TOKEN)

In [ ]:
# ---------------------------------------------------------------------------- #
#                                     SETUP                                    #
# ---------------------------------------------------------------------------- #
PROJECT_NAME='Cushion'
PATH_IN_DRIVE='/gdrive/MyDrive/MyVoices'


def init_project_directory():
    # Create main output directory if it doesn't exist
    out_dir = os.path.join(PATH_IN_DRIVE, 'out')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir,exist_ok=True)


    proj_dir = os.path.join(out_dir, PROJECT_NAME)
    if not os.path.exists(proj_dir):
        os.makedirs(proj_dir,exist_ok=True)
    return proj_dir

def create_sentence_output_directory(i,proj_dir,sentence):
    # Create conversation-based subdirectory
    conversation_text=sentence.lower()
    conversation_text=re.sub('[^a-zA-Z0-9 \n]', '', conversation_text).replace(' ', '_')[:30]
    # Create timestamped subdirectory
    run_dir = os.path.join(proj_dir, f'{i}_{conversation_text}')

    os.makedirs(run_dir,exist_ok=True)

    return run_dir

proj_dir = init_project_directory()
# ------------------------------------- - ------------------------------------ #

In [ ]:
import torch
from generator import Segment,load_csm_1b
import torchaudio
import os
import re
from datetime import datetime
import os

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

def load_audio(audio_path):
    audio_tensor, sample_rate = torchaudio.load(audio_path)
    # Convert to mono by taking mean across channels if needed
    if audio_tensor.dim() > 1 and audio_tensor.size(0) > 1:
        audio_tensor = audio_tensor.mean(dim=0)
    # Resample to match generator sample rate
    audio_tensor = torchaudio.functional.resample(
        audio_tensor, orig_freq=sample_rate,
        new_freq=generator.sample_rate
    )
    # Ensure the tensor is 1D
    if audio_tensor.dim() > 1:
        audio_tensor = audio_tensor.squeeze()
    return audio_tensor

generator = load_csm_1b(device=device)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.21G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

...

enc_c.ckpt:   0%|          | 0.00/185k [00:00<?, ?B/s]

opt.ckpt:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/silentcipher/server.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.enc_c.load_state_dict(self.convert_dataparallel_to_nor

In [ ]:
#input voice
SPEAKER_PROMPTS=[
    {"speaker":0,"audio_path":f'{PATH_IN_DRIVE}/data/original.ogg.acc_segment_1.wav' , "transcript":"I wasn’t even planning to go out, but the sunset looked way too good to miss, so I grabbed my phone and took like a hundred pics"},
    {"speaker":0,"audio_path":f'{PATH_IN_DRIVE}/data/original.ogg.acc_segment_2.wav' , "transcript":"ust tried that viral mac and cheese recipe and oh my god it actually turned out better than I expected!"},
    {"speaker":0,"audio_path":f'{PATH_IN_DRIVE}/data/original.ogg.acc_segment_3.wav' , "transcript":"Woke up super early today by accident, but the quiet morning coffee hit different, not gonna lie."},
    {"speaker":0,"audio_path":f'{PATH_IN_DRIVE}/data/original.ogg.acc_segment_4.wav' , "transcript":"My dog literally barked at his own reflection for ten minutes straight—I couldn’t stop laughing."},
    {"speaker":0,"audio_path":f'{PATH_IN_DRIVE}/data/original.ogg.acc_segment_5.wav' , "transcript":"I got this cute little lamp from a thrift store for five bucks and it totally changed the vibe of my room."},
    ]

#output voice
conversation = [
    {"id":0,"text": "This is an egg...and this is Egg Sitter, the softest pillow ever!", "speaker_id": 0},
    {"id":1,"text": "You sit down...and the egg? It does NOT break!", "speaker_id": 0},
    {"id":2,"text": "Happy back, egg intact.", "speaker_id": 0},
    {"id":3,"text": "Egg Sitter is pure magic!", "speaker_id": 0},
    {"id":4,"text": "Check this out!", "speaker_id": 0},
]

In [ ]:
# Generate each utterance
generated_segments = []

transcripts=[ item['transcript'] for item in SPEAKER_PROMPTS]
audio_paths=[ item['audio_path'] for item in SPEAKER_PROMPTS]
speakers=[ item['speaker'] for item in SPEAKER_PROMPTS]
segments = [
    Segment(text=transcript, speaker=speaker, audio=load_audio(audio_path))
    for transcript, speaker, audio_path in zip(transcripts, speakers, audio_paths)
]

for i,utterance in enumerate(conversation):
    print(f"Generating: {utterance['text']}")
    audio_tensor = generator.generate(
        text=utterance['text'],
        speaker=utterance['speaker_id'],
        context=segments,
        max_audio_length_ms=100_000,
    )
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    generated_segments.append(Segment(text=utterance['text'], speaker=utterance['speaker_id'], audio=audio_tensor))
    output_dir = create_sentence_output_directory(utterance['id'],proj_dir,utterance['text'])
    torchaudio.save(os.path.join(output_dir, f"{timestamp}.wav"), audio_tensor.unsqueeze(0).cpu(), generator.sample_rate)

Generating: Check this out. I'll leave the link below.


In [ ]:
print(f"Successfully generated audio files in: {output_dir}")